# Getting into the flow of TensorFlow

Jupyter notebooks are simply runtime blocks of python code that can interact with eachother. It's a great tool if you're just trying some stuff out with python, you're trying to document a process, or you want an easy to use and visual system to run your python code.

They are composed of markdown blocks and code blocks. This allows for easy stylized documentation.

Code blocks are executed with the "play" button to the left of the block where the brackets are. Once a block is executed, the output stays in memory and any prints, graphs, images, or tables are saved to the notebook itself. This means that a variable or function defined in another block will be visible to all blocks once the block has been executed. This is both useful and can be a pain since everything that's outside of a function global, as if this is one large python script.

You can always rerun a block of code if you're changed things. The outputs will be overwritten and related variables updated.

## Importing libraries

Libraries can be imported anywhere within your code, but often times, people do so at the top of script so that you have everything you need when working.

You can import libraries as is with a simple `import library`. But often it's useful to abreviate so that you don't need to type out the full name like `import lib`. You can also import a function or sub-library like:
`import lib.sublib as slib` or `from lib.sublib import function as func` which will let you use `func()` anwhere in your code then.

In [ ]:
import tensorflow as tf

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

## Datasets


When doing machine learning, you need a dataset to work with for training and testing. This can be images, text, datapoints in some format like csv, etc. In this case, we will be importing the MNIST handwriting dataset where we will try to build a recognizer that will recognize different handwritten numbers as their respective number.

We first import the data set and then during the load step, we get a split of the data with training samples and testing samples. This is normally an 80/20 split.

The X matrix is composed of "features" which describe the data and the Y matrix (or vector) is composed of "labels" which describe what feature we are trying to predict.

In this case, we are working with images that have been converted to matrices of pixels. By default, the values are white color values from 0 to 255 where 0 is black and 255 is white. Everything in between is a form of grey.

We load the data into X and Y training sets, both are subsets of the full set. We also get an X and Y test set which is a smaller subset of the full set. Both of these sets are independent, they have no overlapping samples which is key to measuring the performance of your model (machine learning system).

We then normalize the color values to be floats of range [0., 1.], allowing tensorflow to work with the data.

In [ ]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

In [ ]:
plt.figure()
plt.imshow(x_train[0])
plt.colorbar()
plt.grid(False)
plt.show()

## Training

A model is a machine learning algorithm with a certain set of parameters and a specific data set it trains on. Once it trains, it has a set of 'weights' which make up the model's parameters which is then used for predicting.

We start by gibing some parameters to tensorflow's keras algorithm.

In [ ]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10)
])

We then train the model on a single input, at the same time getting predictions on what class it may be

In [ ]:
predictions = model(x_train[:1]).numpy()
predictions

We then use softmax regression as our activation function to classify the predictions

In [ ]:
tf.nn.softmax(predictions).numpy()

We get a loss function to test our training loss for an itteration

In [ ]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

We then compare our prediction to the expected class and get a loss or error

In [ ]:
loss_fn(y_train[:1], predictions).numpy()

Now we compile our model to make it ready for training

In [ ]:
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

Now we train the full set

In [ ]:
model.fit(x_train, y_train, epochs=5)

Here we evaluate on the test set, the accuracy and loss are shown.

In [ ]:
model.evaluate(x_test,  y_test, verbose=2)

In [ ]:
probability_model = tf.keras.Sequential([
  model,
  tf.keras.layers.Softmax()
])

In [ ]:
probability_model(x_test[:5])